In [67]:
!pwd

/content


In [68]:
!nvidia-smi

Tue Apr 29 22:23:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   64C    P0             30W /   72W |       3MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [69]:
# Connet to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [70]:
!cd '/content/drive/MyDrive/CLIP_video_training'

In [71]:
!git clone https://github.com/sijiasiga/CLIP_video.git '/content/drive/MyDrive/CLIP_video_training/CLIP_video'
# !git clone https://github.com/sijiasiga/CLIP_video.git

Cloning into '/content/drive/MyDrive/CLIP_video_training/CLIP_video'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 230 (delta 29), reused 0 (delta 0), pack-reused 187 (from 1)
Receiving objects: 100% (230/230), 1.51 MiB | 28.58 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [72]:
# From CLIP
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install opencv-python boto3 requests pandas

/bin/bash: line 1: conda: command not found


In [73]:
# # Comment if the file exits
# !wget -O /content/drive/MyDrive/Data/msrvtt_data.zip https://github.com/ArrowLuo/CLIP4Clip/releases/download/v0.0/msrvtt_data.zip
# !unzip /content/drive/MyDrive/Data/msrvtt_data.zip -d /content/drive/MyDrive/Data/

In [74]:
DATA_PATH = "/content/drive/MyDrive/CLIP_video_training/Data/msrvtt_data"  # Directory of MSRVTT data
VIDEO_PATH = "/content/drive/MyDrive/CLIP_video_training/Data/video"    # Directory of MSRVTT raw video

In [81]:
import pandas as pd

# Load training dataset
train_path = f"{DATA_PATH}/MSRVTT_train.9k.csv"
df = pd.read_csv(train_path)

print("Size: ", len(df))
print("Column names", df.columns.tolist())
display(df.head())

# Choose a subset
def sample_subset(df, frac=0.1, random_state=42):
    return df.sample(frac=frac, random_state=random_state).reset_index(drop=True)

# Set fraction to 0.01
subset_df = sample_subset(df, frac=0.1)
print("Size of subset：", len(subset_df))
display(subset_df.head())

# Save the subset
subset_path = f"{DATA_PATH}/MSRVTT_train.subset.csv"
subset_df.to_csv(subset_path, index=False)

print("Subset CSV saved to:", subset_path)


Size:  9000
Column names ['video_id']


,video_id
0,video0
1,video1
2,video2
3,video3
4,video4


Size of subset： 900


,video_id
0,video8405
1,video1162
2,video582
3,video4081
4,video9139


Subset CSV saved to: /content/drive/MyDrive/CLIP_video_training/Data/msrvtt_data/MSRVTT_train.subset.csv


In [75]:
import torch
print("Current device:", torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU only")

Current device: NVIDIA L4


In [78]:
!wget -P /content/drive/MyDrive/CLIP_video_training/CLIP_video/modules https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt

--2025-04-29 22:23:46--  https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.59, 2620:1ec:bdf::59
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353976522 (338M) [application/octet-stream]
Saving to: ‘/content/drive/MyDrive/CLIP_video_training/CLIP_video/modules/ViT-B-32.pt’

ViT-B-32.pt         100%[===================>] 337.58M  48.9MB/s    in 6.6s    

2025-04-29 22:23:53 (51.3 MB/s) - ‘/content/drive/MyDrive/CLIP_video_training/CLIP_video/modules/ViT-B-32.pt’ saved [353976522/353976522]



In [79]:
torch.cuda.empty_cache()

In [ ]:
# # Updata changes to files. Comment if the files are not changed
# !cd CLIP_video && git pull

In [ ]:
# !pip install ffmpeg
# !python /content/CLIP_video/preprocess/compress_video.py --input_root "/content/drive/MyDrive/IDL/IDL Project/Data/video" --output_root "/content/drive/MyDrive/IDL/IDL Project/Data/compressed_video"

In [84]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

!python /content/drive/MyDrive/CLIP_video_training/CLIP_video/main_task_retrieval.py \
  --do_train \
  --num_thread_reader 8 \
  --epochs 20 \
  --batch_size 64 \
  --train_csv  "$DATA_PATH"/MSRVTT_train.subset.csv \
  --val_csv    "$DATA_PATH"/MSRVTT_JSFUSION_test.csv \
  --data_path  "$DATA_PATH"/MSRVTT_data.json \
  --features_path "$VIDEO_PATH" \
  --output_dir /content/drive/MyDrive/CLIP_video_training/CLIP_video/ckpts/ckpt_msrvtt_retrieval_looseType \
  --lr 1e-4 \
  --max_words 32 \
  --max_frames 8 \
  --batch_size_val 8 \
  --datatype msrvtt \
  --expand_msrvtt_sentences \
  --feature_framerate 1 \
  --coef_lr 1e-3 \
  --freeze_layer_num 6 \
  --slice_framepos 2 \
  --loose_type \
  --linear_patch 2d \
  --sim_header meanP \
  --pretrained_clip_name ViT-B/32 \
  --fp16 \
  --fp16_opt_level O1


04/29/2025 22:33:55 - INFO -   Effective parameters:
04/29/2025 22:33:55 - INFO -     <<< batch_size: 64
04/29/2025 22:33:55 - INFO -     <<< batch_size_val: 8
04/29/2025 22:33:55 - INFO -     <<< cache_dir: 
04/29/2025 22:33:55 - INFO -     <<< coef_lr: 0.001
04/29/2025 22:33:55 - INFO -     <<< cross_model: cross-base
04/29/2025 22:33:55 - INFO -     <<< cross_num_hidden_layers: 4
04/29/2025 22:33:55 - INFO -     <<< data_path: /content/drive/MyDrive/CLIP_video_training/Data/msrvtt_data/MSRVTT_data.json
04/29/2025 22:33:55 - INFO -     <<< datatype: msrvtt
04/29/2025 22:33:55 - INFO -     <<< do_eval: False
04/29/2025 22:33:55 - INFO -     <<< do_lower_case: False
04/29/2025 22:33:55 - INFO -     <<< do_pretrain: False
04/29/2025 22:33:55 - INFO -     <<< do_train: True
04/29/2025 22:33:55 - INFO -     <<< epochs: 20
04/29/2025 22:33:55 - INFO -     <<< eval_frame_order: 0
04/29/2025 22:33:55 - INFO -     <<< expand_msrvtt_sentences: True
04/29/2025 22:33:55 - INFO -     <<< feature_